In [1]:
from Microdados_CAGED import *
import pandas as pd

In [2]:
# Aqui cria-se dois data.frames: os dados brutos e os sumarizados pelo pandas
caged_microdados, caged_agrupamentos = importar_caged_mes_ano(ano_base=2023, mes_base=8)

In [3]:
# Verificação se dos itens criados, temos de fato um data.frame
if isinstance(caged_agrupamentos["município"], pd.DataFrame):
    print("É um DataFrame")
else:
    print("Não é um DataFrame")

É um DataFrame


In [4]:
caged_agrupamentos["município"] # Verificando um dos itens do dicionário

Desligamentos  Admissões Salario_medio  Saldo
Período    município                                               
2023-08-01 220005                 2          3      59927.64      1
           220010                 1          3        846.45      2
           220020                24         47       1368.36     23
           220025                 2          1       1697.95     -1
           220030                17          8       1402.06     -9
...                             ...        ...           ...    ...
           221110                84        122       1415.09     38
           221120               146        249       1644.27    103
           221130                41         30       1425.48    -11
           221150                 2          0       1333.88     -2
           221160                 0          2       2277.27      2

[187 rows x 4 columns]

In [5]:
caged_dimensoes = importar_dimensoes() # o dicionário de tabelas-dimensões

In [6]:
# O dicionário de dos dados agrupados, separados pelo respectivo grupo
# Esta etapa é apenas para separar os objetos, deixando salvos os objtos acima
 
dataframes_dict = {}
for key in pages[:-1]:
    dataframes_dict[key] = pd.DataFrame(caged_agrupamentos[key])

# temos um dicionário de DataFrames separados
dataframes_dict

{'município':                       Desligamentos  Admissões Salario_medio  Saldo
 Período    município                                               
 2023-08-01 220005                 2          3      59927.64      1
            220010                 1          3        846.45      2
            220020                24         47       1368.36     23
            220025                 2          1       1697.95     -1
            220030                17          8       1402.06     -9
 ...                             ...        ...           ...    ...
            221110                84        122       1415.09     38
            221120               146        249       1644.27    103
            221130                41         30       1425.48    -11
            221150                 2          0       1333.88     -2
            221160                 0          2       2277.27      2
 
 [187 rows x 4 columns],
 'subclasse':                       Desligamentos  Admissões Sa

In [7]:
# Consolidação dos resultados, conforme os parâmetros da função que criamos
caged_formatado_completo = consolidar_caged(caged_agrupamentos, caged_dimensoes)

In [8]:
caged_formatado_completo["graudeinstrução"] # verificando a disposição de um dos itens

,Período,graudeinstrução,Admissões,Desligamentos,Saldo,Salário médio
0,2023-08-01,Analfabeto,63,64,-1,3680.66
1,2023-08-01,Até 5ª Incompleto,276,308,-32,1591.65
2,2023-08-01,5ª Completo Fundamental,228,181,47,1567.59
3,2023-08-01,6ª a 9ª Fundamental,725,520,205,2028.86
4,2023-08-01,Fundamental Completo,1453,1024,429,1923.00
5,2023-08-01,Médio Incompleto,876,736,140,2552.65
6,2023-08-01,Médio Completo,8166,6478,1688,2603.13
7,2023-08-01,Superior Incompleto,416,405,11,2813.22
8,2023-08-01,Superior Completo,1144,926,218,4304.46
9,2023-08-01,Mestrado,62,39,23,2134.29


In [9]:
# Inserir análise dos salários no nosso dicionário
df_salários = analisar_salarios(caged_microdados, caged_dimensoes, "município")
caged_formatado_completo["salário"] = df_salários

      município    IBGE7                UF              Descrição  \
0        110001  1100015                RO  Alta Floresta D´oeste   
1        110002  1100023                RO              Ariquemes   
2        110003  1100031                RO                 Cabixi   
3        110004  1100049                RO                 Cacoal   
4        110005  1100056                RO             Cerejeiras   
...         ...      ...               ...                    ...   
5566     522205  5222054                GO         Vicentinópolis   
5567     522220  5222203                GO               Vila Boa   
5568     522230  5222302                GO          Vila Propício   
5569     530010  5300108                DF               Brasília   
5570     999999  9999999  Não identificado       Não identificado   

                   Região Unnamed: 5  
0            Região Norte        NaN  
1            Região Norte        NaN  
2            Região Norte        NaN  
3            Re

In [10]:
# salvando os arquivos
#salvar_arquivos(caged_formatado_completo, "CAGED_municípios")

In [60]:
def analises_combinadass(df_microdados, caged_dimensoes, grupos = []):
    #tr
    df = df_microdados
    caged_formatado = df.groupby(['Período', *grupos]).agg(
            
            Admissões=('saldomovimentação', lambda x: (x == 1).sum()),
            Desligamentos=('saldomovimentação', lambda x: (x == -1).sum())
        )
    
    caged_formatado["Saldo"] = caged_formatado["Admissões"] - caged_formatado["Desligamentos"]
    for categoria in grupos:
        if categoria != "Setores":
            df_cat_ref = caged_dimensoes[categoria]
            
            # Renomeie as colunas do DataFrame da direita (df_cat_ref)
            df_cat_ref = df_cat_ref.rename(columns={"Código": categoria})
            
             # Renomeie as colunas do DataFrame da direita (df_cat_ref)
            df_cat_ref = df_cat_ref.rename(columns={"Código": categoria, "Descrição": f"{categoria}_Descrição"})
            
            # Realize o join e mantenha apenas as colunas desejadas
            caged_formatado = caged_formatado.join(df_cat_ref.set_index(categoria)[[f"{categoria}_Descrição"]], on=categoria)

              
    return caged_formatado
        

In [63]:
# teste da analise combinada
df_detalhamento = analises_combinadass(caged_microdados, caged_dimensoes, grupos=["Setores", "sexo"])

df_detalhamento.to_excel("./Tabelas/Análise_combinada.xlsx")